# Affect Modeling LM

This is the model file for linear models

### Model 1

In [ ]:
data_model_m1=list(
  nTrials = length(data$Rating),
  nSubject = length(unique(data$Sub)),
  subject = data$Sub,
  o.true = data$outcome.true,
  y = data$Rating,
  y_previous = data$Rating_previous)



model_String_m1 ='
model {

  #likelihood function
  for (t in 1:nTrials) {
      
      y[t] ~ dnorm(beta0[subject[t]] + beta1[subject[t]] * o.true[t] + beta_previous[subject[t]] *y_previous[t], precy)
  }
  
  #looic
  
  for (t in 1:nTrials) {
     
   
      ld[t] = logdensity.norm(y[t], beta0[subject[t]] + beta1[subject[t]] * o.true[t] + beta_previous[subject[t]] *y_previous[t], precy)
  }
  
    
  #priors
  for (i in 1:nSubject) {
    beta_previous[i] ~ dnorm(mubeta_previous, precbeta_previous)
    beta0[i] ~ dnorm(mubeta0, precbeta0)
    beta1[i] ~ dnorm(mubeta1, precbeta1)
    
  }

  #hyperpriors
   
  mubeta_previous ~ dnorm(0, 1e-3)
  mubeta0 ~ dnorm(0, 1e-3)
  mubeta1 ~ dnorm(0, 1e-3)

  
  precbeta_previous ~ dgamma(.001, .001)
  precbeta0 ~ dgamma(.001, .001)
  precbeta1 ~ dgamma(.001, .001)
 
  precy ~ dgamma(.001, .001)
}

'

writeLines(model_String_m1, con = 'Model 2.txt')

#init function
initfunction_m1 <- function(chain){
  return(list(
    mubeta0 = runif(1, 20, 80),
    mubeta1 = runif(1, 10, 20),
    mubeta_previous = runif(1, -0.8, 0.8),
    precbeta0 = runif(1, 1, 100),
    precbeta1 = runif(1, 1, 100),
    precbeta_previous =  runif(1, 1, 100),
    #y = matrix(NA,nrow = 192, ncol = data_model$nSubject),
    .RNG.name = "lecuyer::RngStream",
    .RNG.seed = sample.int(1e10, 1, replace = F)))
}



#Create list of parameters to be monitored
parameters_m1 <- c("beta0", "beta1", "beta_previous", 
                   "mubeta0", "mubeta1", "mubeta_previous",
                   "precbeta0", "precbeta1", "precbeta_previous", "ld")




#Run the model in runjags
startTime = proc.time()
jagsModel_m1 <- run.jags(method = "parallel",
                         model = 'Model 2.txt',
                         monitor = parameters_m1,
                         data = data_model_m1,
                         inits = initfunction_m1,
                         n.chains = 3,
                         adapt = 2000, #how long the samplers "tune"
                         burnin = 2000, #how long of a burn in
                         sample = 1000,
                         thin = 3, #thin if high autocorrelation to avoid huge files
                         modules = c("lecuyer"),
                         summarise = F,
                         plots = T)
stopTime = proc.time()
elapsedTime = stopTime - startTime
show(elapsedTime/60) #Tells how long it took to run analysis

save(jagsModel_m1, file = "jags_sample_M1.RData")

### Model 2

In [ ]:
data_model_m2=list(
  nTrials = length(data$Rating),
  nSubject = length(unique(data$Sub)),
  subject = data$Sub,
  o.true = data$outcome.true,
  o.abs = abs(data$outcome.true),
  y = data$Rating,
  y_previous = data$Rating_previous)


model_String_m2 ='
model {

  #likelihood function
  for (t in 1:nTrials) {
      
      #create a variable that reflect the true outcome
      o[t] = ifelse(o.true[t]>0, 1, 2)
      
      y[t] ~ dnorm(beta0[subject[t],o[t]] + beta1[subject[t],o[t]] * o.abs[t] + beta_previous[subject[t]] *y_previous[t], precy)
  }
  
  #looic
  
  for (t in 1:nTrials) {
     
   
      ld[t] = logdensity.norm(y[t], beta0[subject[t],o[t]] + beta1[subject[t],o[t]] * o.abs[t] + beta_previous[subject[t]] *y_previous[t], precy)
  }
  
    
  #priors
  for (i in 1:nSubject) {
    beta_previous[i] ~ dnorm(mubeta_previous, precbeta_previous)
    beta0[i,1] ~ dnorm(mubeta0w, precbeta0w)
    beta1[i,1] ~ dnorm(mubeta1w, precbeta1w)
    beta0[i,2] ~ dnorm(mubeta0l, precbeta0l)
    beta1[i,2] ~ dnorm(mubeta1l, precbeta1l)
    
  }

  #hyperpriors
   
  mubeta_previous ~ dnorm(0, 1e-3)
  mubeta0w ~ dnorm(0, 1e-3)
  mubeta1w ~ dnorm(0, 1e-3)
  mubeta0l ~ dnorm(0, 1e-3)
  mubeta1l ~ dnorm(0, 1e-3)

  
  precbeta_previous ~ dgamma(.001, .001)
  precbeta0w ~ dgamma(.001, .001)
  precbeta1w ~ dgamma(.001, .001)
  precbeta0l ~ dgamma(.001, .001)
  precbeta1l ~ dgamma(.001, .001)
 
  precy ~ dgamma(.001, .001)
}

'

writeLines(model_String_m2, con = 'Model 2.txt')

#init function
initfunction_m2 <- function(chain){
  return(list(
    mubeta0w = runif(1, 20, 80),
    mubeta1w = runif(1, 10, 20),
    mubeta0l = runif(1, 20, 80),
    mubeta1l = runif(1, 10, 20),
    mubeta_previous = runif(1, -0.8, 0.8),
    precbeta0w = runif(1, 1, 100),
    precbeta1w = runif(1, 1, 100),
    precbeta0l = runif(1, 1, 100),
    precbeta1l = runif(1, 1, 100),
    precbeta_previous =  runif(1, 1, 100),
    #y = matrix(NA,nrow = 192, ncol = data_model$nSubject),
    .RNG.name = "lecuyer::RngStream",
    .RNG.seed = sample.int(1e10, 1, replace = F)))
}



#Create list of parameters to be monitored
parameters_m2 <- c("beta0", "beta1", "beta_previous", 
                   "mubeta0w", "mubeta1w", "mubeta0l", "mubeta1l", "mubeta_previous",
                   "precbeta0w", "precbeta1w","precbeta0l", "precbeta1l", "precbeta_previous", "ld")




#Run the model in runjags
startTime = proc.time()
jagsModel_m2 <- run.jags(method = "parallel",
                         model = 'Model 2.txt',
                         monitor = parameters_m2,
                         data = data_model_m2,
                         inits = initfunction_m2,
                         n.chains = 3,
                         adapt = 2000, #how long the samplers "tune"
                         burnin = 2000, #how long of a burn in
                         sample = 1000,
                         thin = 3, #thin if high autocorrelation to avoid huge files
                         modules = c("wiener", "lecuyer"),
                         summarise = F,
                         plots = T)
stopTime = proc.time()
elapsedTime = stopTime - startTime
show(elapsedTime/60) #Tells how long it took to run analysis

#save(jagsModel_m2, file = "jags_sample_M2.RData")

### Model 3

In [ ]:
data_model_m3=list(
  nTrials = length(data$Rating),
  nSubject = length(unique(data$Sub)),
  subject = data$Sub,
  o.true = data$outcome.true,
  o.abs = abs(data$outcome.true),
  expectedness = data$expectedness,
  y = data$Rating,
  y_previous = data$Rating_previous)



model_String_m3 ='
model {

  #likelihood function
  for (t in 1:nTrials) {
      
    o[t] = ifelse(o.true[t]>0, 1, 2)

    y[t] ~ dnorm(beta0[subject[t],o[t]] + beta1[subject[t],o[t]] * o.abs[t] + beta2[subject[t],o[t]]*expectedness[t]
      + beta3[subject[t],o[t]]* o.abs[t] *expectedness[t] + beta_previous[subject[t]] *y_previous[t], precy)
  }
  
  #looic
  
  for (t in 1:nTrials) {
     
   
      ld[t] = logdensity.norm(y[t], beta0[subject[t],o[t]] + beta1[subject[t],o[t]] * o.abs[t] + beta2[subject[t],o[t]]*expectedness[t]
      + beta3[subject[t],o[t]] * o.abs[t] *expectedness[t] + beta_previous[subject[t]] *y_previous[t], precy)
  }
  
    
  #priors
  for (i in 1:nSubject) {
    beta_previous[i] ~ dnorm(mubeta_previous, precbeta_previous)
    beta0[i,1] ~ dnorm(mubeta0w, precbeta0w)
    beta1[i,1] ~ dnorm(mubeta1w, precbeta1w)
    beta2[i,1] ~ dnorm(mubeta2w, precbeta2w)
    beta3[i,1] ~ dnorm(mubeta3w, precbeta3w)
    beta0[i,2] ~ dnorm(mubeta0l, precbeta0l)
    beta1[i,2] ~ dnorm(mubeta1l, precbeta1l)
    beta2[i,2] ~ dnorm(mubeta2l, precbeta2l)
    beta3[i,2] ~ dnorm(mubeta3l, precbeta3l)
    
  }

  #hyperpriors
   
  mubeta_previous ~ dnorm(0, 1e-3)
  mubeta0w ~ dnorm(0, 1e-3)
  mubeta1w ~ dnorm(0, 1e-3)
  mubeta2w ~ dnorm(0, 1e-3)
  mubeta3w ~ dnorm(0, 1e-3)
  mubeta0l ~ dnorm(0, 1e-3)
  mubeta1l ~ dnorm(0, 1e-3)
  mubeta2l ~ dnorm(0, 1e-3)
  mubeta3l ~ dnorm(0, 1e-3)
  
  
  precbeta_previous ~ dgamma(.001, .001)
  precbeta0w ~ dgamma(.001, .001)
  precbeta1w ~ dgamma(.001, .001)
  precbeta2w ~ dgamma(.001, .001)
  precbeta3w ~ dgamma(.001, .001)
  precbeta0l ~ dgamma(.001, .001)
  precbeta1l ~ dgamma(.001, .001)
  precbeta2l ~ dgamma(.001, .001)
  precbeta3l ~ dgamma(.001, .001)
  
  precy ~ dgamma(.001, .001)
}

'

writeLines(model_String_m3, con = 'Model 3.txt')

#init function
initfunction_m3 <- function(chain){
  return(list(
    mubeta0w = runif(1, 20, 80),
    mubeta1w = runif(1, 10, 20),
    mubeta2w = runif(1, 10, 20),
    mubeta3w = runif(1, 10, 20),
    mubeta0l = runif(1, 20, 80),
    mubeta1l = runif(1, 10, 20),
    mubeta2l = runif(1, 10, 20),
    mubeta3l = runif(1, 10, 20),
    mubeta_previous = runif(1, -0.8, 0.8),
    precbeta0w = runif(1, 1, 100),
    precbeta1w = runif(1, 1, 100),
    precbeta2w = runif(1, 1, 100),
    precbeta3w = runif(1, 1, 100),
    precbeta0l = runif(1, 1, 100),
    precbeta1l = runif(1, 1, 100),
    precbeta2l = runif(1, 1, 100),
    precbeta3l = runif(1, 1, 100),
    precbeta_previous =  runif(1, 1, 100),
    .RNG.name = "lecuyer::RngStream",
    .RNG.seed = sample.int(1e10, 1, replace = F)))
}



#Create list of parameters to be monitored
parameters_m3 <- c("beta0", "beta1", "beta2", "beta3", "beta_previous", 
                "mubeta0w", "mubeta1w", "mubeta2w", "mubeta3w",
                "mubeta0l", "mubeta1l", "mubeta2l", "mubeta3l","mubeta_previous",
                "precbeta0w", "precbeta1w","precbeta2w", "precbeta3w", 
                "precbeta0l", "precbeta1l","precbeta2l", "precbeta3l", "precbeta_previous", "ld")




#Run the model in runjags
startTime = proc.time()
jagsModel_m3 <- run.jags(method = "parallel",
                      model = 'Model 3.txt',
                      monitor = parameters_m3,
                      data = data_model_m3,
                      inits = initfunction_m3,
                      n.chains = 3,
                      adapt = 2000, #how long the samplers "tune"
                      burnin = 2000, #how long of a burn in
                      sample = 1000,
                      thin = 3, #thin if high autocorrelation to avoid huge files
                      modules = c("lecuyer"),
                      summarise = F,
                      plots = T)
stopTime = proc.time()
elapsedTime = stopTime - startTime
show(elapsedTime/60) #Tells how long it took to run analysis

save(jagsModel_m3, file = "jags_sample_M3.RData")


### Model 4

In [ ]:

data_model_m4=list(
  nTrials = length(data$Rating),
  nSubject = length(unique(data$Sub)),
  subject = data$Sub,
  o.true = data$outcome.true,
  o.win = data$outcome.win,
  o.loss = data$outcome.loss,
  p.win = data$prob.win,
  p.loss = data$prob.loss,
  y = data$Rating,
  y_previous = data$Rating_previous)



model_String_m4 ='
model {

  #likelihood function
  for (t in 1:nTrials) {
      
      y[t] ~ dnorm(beta0[subject[t]] + beta1[subject[t]] * (o.true[t] - (gamma[subject[t]] *p.win[t]*o.win[t] 
                          + (2-gamma[subject[t]])*p.loss[t]*o.loss[t])) + beta_previous[subject[t]] * y_previous[t], precy)
  }
  
  #looic
  
  for (t in 1:nTrials) {
     
   
      ld[t] = logdensity.norm(y[t], beta0[subject[t]] + beta1[subject[t]] * (o.true[t] - (gamma[subject[t]] *p.win[t]*o.win[t] 
                          + (2-gamma[subject[t]])*p.loss[t]*o.loss[t])) + beta_previous[subject[t]] * y_previous[t], precy)
  }
  
    
  #priors
  for (i in 1:nSubject) {
    gamma[i] = 2 * (1/(1+exp(-logit_gamma[i])))
    beta_previous[i] ~ dnorm(mubeta_previous, precbeta_previous)
    beta0[i] ~ dnorm(mubeta0, precbeta0)
    beta1[i] ~ dnorm(mubeta1, precbeta1)
    logit_gamma[i] ~ dnorm(mugamma, precgamma)
    
  }

  #hyperpriors
   
  mubeta_previous ~ dnorm(0, 1e-3)
  mubeta0 ~ dnorm(0, 1e-3)
  mubeta1 ~ dnorm(0, 1e-3)
  mugamma ~ dnorm(0, 1e-3)
  
  precbeta_previous ~ dgamma(.001, .001)
  precbeta0 ~ dgamma(.001, .001)
  precbeta1 ~ dgamma(.001, .001)
  
  precgamma ~ dgamma(.001,.001)
  precy ~ dgamma(.001, .001)
}

'

writeLines(model_String_m4, con = 'Model 4.txt')

#init function
initfunction_m4 <- function(chain){
  return(list(
    mubeta0 = runif(1, 20, 80),
    mubeta1 = runif(1, 10, 20),
    mubeta_previous = runif(1, -0.8, 0.8),
    mugamma = runif(1, 0.01, 1.89),
    precbeta0 = runif(1, 1, 100),
    precbeta1 = runif(1, 1, 100),
    precbeta_previous = runif(1, 1, 100),
    precgamma = runif(1, 1, 100),
    .RNG.name = "lecuyer::RngStream",
    .RNG.seed = sample.int(1e10, 1, replace = F)))
}



#Create list of parameters to be monitored
parameters_m4 <- c("beta_previous", "beta0", "beta1", "gamma",
                "mubeta_previous", "mubeta0", "mubeta1", "mugamma",
                "precbeta_previous", "precbeta0", "precbeta1", "precgamma", "ld")




#Run the model in runjags
startTime = proc.time()
jagsModel_m4 <- run.jags(method = "parallel",
                      model = 'Model 4.txt',
                      monitor = parameters_m4,
                      data = data_model_m4,
                      inits = initfunction_m4,
                      n.chains = 3,
                      adapt = 2000, #how long the samplers "tune"
                      burnin = 2000, #how long of a burn in
                      sample = 1000,
                      thin = 3, #thin if high autocorrelation to avoid huge files
                      modules = c("lecuyer"),
                      summarise = F,
                      plots = T)
stopTime = proc.time()
elapsedTime = stopTime - startTime
show(elapsedTime/60) #Tells how long it took to run analysis

save(jagsModel_m4, file = "jags_sample_M4.RData")

### Model 4*

In [ ]:

data_model_m4star=list(
  nTrials = length(data$Rating),
  nSubject = length(unique(data$Sub)),
  subject = data$Sub,
  o.true = data$outcome.true,
  o.win = data$outcome.win,
  o.loss = data$outcome.loss,
  p.win = data$prob.win,
  p.loss = data$prob.loss,
  y = data$Rating,
  y_previous = data$Rating_previous)



model_String_m4star ='
model {

  #likelihood function
  for (t in 1:nTrials) {
      
      y[t] ~ dnorm(beta0[subject[t]] + beta1[subject[t]] * (o.true[t] - (p.win[t]*o.win[t] 
                          + *p.loss[t]*o.loss[t])) + beta_previous[subject[t]] * y_previous[t], precy)
  }
  
  #looic
  
  for (t in 1:nTrials) {
     
   
      ld[t] = logdensity.norm(y[t], beta0[subject[t]] + beta1[subject[t]] * (o.true[t] - (p.win[t]*o.win[t] 
                          +p.loss[t]*o.loss[t])) + beta_previous[subject[t]] * y_previous[t], precy)
  }
  
    
  #priors
  for (i in 1:nSubject) {
    beta_previous[i] ~ dnorm(mubeta_previous, precbeta_previous)
    beta0[i] ~ dnorm(mubeta0, precbeta0)
    beta1[i] ~ dnorm(mubeta1, precbeta1)
  }

  #hyperpriors
   
  mubeta_previous ~ dnorm(0, 1e-3)
  mubeta0 ~ dnorm(0, 1e-3)
  mubeta1 ~ dnorm(0, 1e-3)
  
  precbeta_previous ~ dgamma(.001, .001)
  precbeta0 ~ dgamma(.001, .001)
  precbeta1 ~ dgamma(.001, .001)
  
  precy ~ dgamma(.001, .001)
}

'

writeLines(model_String_m4star, con = 'Model 4star.txt')

#init function
initfunction_m4star <- function(chain){
  return(list(
    mubeta0 = runif(1, 20, 80),
    mubeta1 = runif(1, 10, 20),
    mubeta_previous = runif(1, -0.8, 0.8),
    mugamma = runif(1, 0.01, 1.89),
    precbeta0 = runif(1, 1, 100),
    precbeta1 = runif(1, 1, 100),
    precbeta_previous = runif(1, 1, 100),
    precgamma = runif(1, 1, 100),
    .RNG.name = "lecuyer::RngStream",
    .RNG.seed = sample.int(1e10, 1, replace = F)))
}



#Create list of parameters to be monitored
parameters_m4star <- c("beta_previous", "beta0", "beta1", 
                   "mubeta_previous", "mubeta0", "mubeta1", 
                   "precbeta_previous", "precbeta0", "precbeta1", "ld")




#Run the model in runjags
startTime = proc.time()
jagsModel_m4star <- run.jags(method = "parallel",
                         model = 'Model 5.txt',
                         monitor = parameters_m4star,
                         data = data_model_m4star,
                         inits = initfunction_m4star,
                         n.chains = 3,
                         adapt = 2000, #how long the samplers "tune"
                         burnin = 2000, #how long of a burn in
                         sample = 1000,
                         thin = 3, #thin if high autocorrelation to avoid huge files
                         modules = c("lecuyer"),
                         summarise = F,
                         plots = T)
stopTime = proc.time()
elapsedTime = stopTime - startTime
show(elapsedTime/60) #Tells how long it took to run analysis

save(jagsModel_m4star, file = "jags_sample_M4star.RData")

### Model 5

In [ ]:

data_model_m5=list(
  nTrials = length(data$Rating),
  nSubject = length(unique(data$Sub)),
  subject = data$Sub,
  o.true = data$outcome.true,
  expectedness = data$expectedness,
  y = data$Rating,
  y_previous = data$Rating_previous)



model_String_m5 ='
model {

  #likelihood function
  for (t in 1:nTrials) {
      
      y[t] ~ dnorm(beta0[subject[t]] + beta1[subject[t]] * o.true[t] * expectedness[t]+ beta_previous[subject[t]] * y_previous[t], precy)
  }
  
  #looic
  
  for (t in 1:nTrials) {
     
   
      ld[t] = logdensity.norm(y[t], beta0[subject[t]] + beta1[subject[t]] * o.true[t] * expectedness[t] + beta_previous[subject[t]] * y_previous[t], precy)
  }
  
    
  #priors
  for (i in 1:nSubject) {
    beta_previous[i] ~ dnorm(mubeta_previous, precbeta_previous)
    beta0[i] ~ dnorm(mubeta0, precbeta0)
    beta1[i] ~ dnorm(mubeta1, precbeta1)
  }

  #hyperpriors
   
  mubeta_previous ~ dnorm(0, 1e-3)
  mubeta0 ~ dnorm(0, 1e-3)
  mubeta1 ~ dnorm(0, 1e-3)
  
  precbeta_previous ~ dgamma(.001, .001)
  precbeta0 ~ dgamma(.001, .001)
  precbeta1 ~ dgamma(.001, .001)
  
  precy ~ dgamma(.001, .001)
}

'

writeLines(model_String_m5, con = 'Model 4star.txt')

#init function
initfunction_m5 <- function(chain){
  return(list(
    mubeta0 = runif(1, 20, 80),
    mubeta1 = runif(1, 10, 20),
    mubeta_previous = runif(1, -0.8, 0.8),
    mugamma = runif(1, 0.01, 1.89),
    precbeta0 = runif(1, 1, 100),
    precbeta1 = runif(1, 1, 100),
    precbeta_previous = runif(1, 1, 100),
    precgamma = runif(1, 1, 100),
    .RNG.name = "lecuyer::RngStream",
    .RNG.seed = sample.int(1e10, 1, replace = F)))
}



#Create list of parameters to be monitored
parameters_m5 <- c("beta_previous", "beta0", "beta1", 
                       "mubeta_previous", "mubeta0", "mubeta1", 
                       "precbeta_previous", "precbeta0", "precbeta1", "ld")




#Run the model in runjags
startTime = proc.time()
jagsModel_m5 <- run.jags(method = "parallel",
                             model = 'Model 5.txt',
                             monitor = parameters_m5,
                             data = data_model_m5,
                             inits = initfunction_m5,
                             n.chains = 3,
                             adapt = 2000, #how long the samplers "tune"
                             burnin = 2000, #how long of a burn in
                             sample = 1000,
                             thin = 3, #thin if high autocorrelation to avoid huge files
                             modules = c("lecuyer"),
                             summarise = F,
                             plots = T)
stopTime = proc.time()
elapsedTime = stopTime - startTime
show(elapsedTime/60) #Tells how long it took to run analysis

save(jagsModel_m5, file = "jags_sample_M5.RData")